In [1]:
import numpy as np

import data_preprocessing
import neural_metrics

In [13]:
data_str = 'military_vehicles'
main_model_name = binary_model_name = additional_model_name = 'vit_b_16'
secondary_model_name = 'vit_l_16'
main_lr = binary_lr = 0.0001
original_num_epochs = secondary_num_epochs = 10
additional_epoch_num = 30
binary_num_epochs = 10
number_of_fine_classes = 24

# data_str = 'imagenet'
# main_model_name = binary_model_name = additional_model_name =  'dinov2_vits14'
# secondary_model_name = 'dinov2_vitl14'
# main_lr = binary_lr = 0.000001
# original_num_epochs = additional_epoch_num = 8
# secondary_num_epochs = 2
# binary_num_epochs = 5
# number_of_fine_classes = 42

# data_str = 'openimage'
# main_model_name = 'tresnet_m'
# secondary_model_name = additional_model_name = binary_model_name = 'dinov2_vits14'
# main_lr = binary_lr = 0.000001
# original_num_epochs = 0
# secondary_num_epochs = 2
# binary_num_epochs = 4
# number_of_fine_classes = 30
# additional_epoch_num = 10

preprocessor = data_preprocessing.DataPreprocessor(data_str=data_str)

additional_results_path = 'combined_results'
fine_test_pred = np.load(f'{additional_results_path}/'
                          f"{f'{data_str}_' if not (data_str == 'military_vehicles' and original_num_epochs == 20) else ''}{main_model_name}_test_fine_pred_BCE_lr{main_lr}_e{original_num_epochs - 1}.npy")
coarse_test_pred = np.load(f'{additional_results_path}/'
                          f"{f'{data_str}_' if not (data_str == 'military_vehicles' and original_num_epochs == 20) else ''}{main_model_name}_test_coarse_pred_BCE_lr{main_lr}_e{original_num_epochs - 1}.npy")
additional_fine_test_pred = np.load(f'{additional_results_path}/'
                          f'{data_str}_{additional_model_name}_'
                                    f'test_fine_pred_BCE_lr{main_lr}_e{additional_epoch_num - 1}_additional.npy')
additional_coarse_test_pred = np.load(f'{additional_results_path}/'
                          f'{data_str}_{additional_model_name}_'
                                      f'test_coarse_pred_BCE_lr{main_lr}_e{additional_epoch_num - 1}_additional.npy')
fine_acc, fine_f1, _, _ = neural_metrics.get_individual_metrics(pred_data=additional_fine_test_pred,
                                                                true_data=preprocessor.test_true_fine_data)
coarse_acc, coarse_f1, _, _ = neural_metrics.get_individual_metrics(pred_data=additional_coarse_test_pred,
                                                          true_data=preprocessor.test_true_coarse_data)
print(f'Additional model fine acc: {fine_acc}, fine_f1: {fine_f1}\n'
      f'Additional model coarse acc: {coarse_acc}, coarse_f1: {coarse_f1}')

Additional model fine acc: 0.6181369524984578, fine_f1: 0.6043744631728198
Additional model coarse acc: 0.8155459592843923, coarse_f1: 0.8009447438202129


In [16]:
error_predictions = (np.where(fine_test_pred != additional_fine_test_pred, 1, 0) | 
                     np.where(coarse_test_pred != additional_coarse_test_pred , 1, 0))
error_ground_truth = (np.where(fine_test_pred != preprocessor.test_true_fine_data , 1, 0) | 
                      np.where(coarse_test_pred != preprocessor.test_true_coarse_data, 1, 0))
error_acc, balanced_accuracy, error_f1, _, _ = neural_metrics.get_individual_metrics(pred_data=error_predictions,
                                                                  true_data=error_ground_truth,
                                                                  binary=True)
print(f'Additional model error acc: {error_acc}\nbalanced accuracy: {balanced_accuracy}\nfine_f1: {error_f1}')

Additional model error acc: 0.8032078963602715
balanced accuracy: 0.8010123385922219
fine_f1: 0.8018128544024958
